In [1]:
import os
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
%matplotlib widget
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    clean_degree_4,
)
import scipy
from amftrack.pipeline.functions.image_processing.node_id import remove_spurs
from amftrack.pipeline.functions.image_processing.extract_skel import remove_component, remove_holes
import numpy as np
from amftrack.pipeline.development.high_mag_videos.high_mag_videos_fun import *
from scipy import signal
from amftrack.pipeline.functions.image_processing.extract_skel import (
    extract_skel_new_prince,
    run_back_sub,
    bowler_hat,
)
from scipy.interpolate import griddata

from skimage.morphology import skeletonize
from amftrack.util.sys import temp_path
import pandas as pd
from PIL import Image
from scipy.optimize import curve_fit
from amftrack.pipeline.functions.image_processing.extract_skel import (
    extract_skel_new_prince,
    run_back_sub,
    bowler_hat,
)




2023-01-04 23:13:00,626-[INFO]- numexpr.utils:148 -> Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-01-04 23:13:00,627-[INFO]- numexpr.utils:160 -> NumExpr defaulting to 8 threads.


/home/ipausers/bisot/pycode/MscThesis/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2023-01-04 23:13:16.816883: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-04 23:13:20.792584: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ipausers/bisot/miniconda3/envs/amftrack/lib/python3.9/site-packages/cv2/../../lib64:
2023-01-04 23:13:20.792604: I tensorflow/stream_executor/cuda/cudart_stub.

In [9]:
directory = r"/mnt/sun-temp/TEMP/Transport/TransportCUT/20220104_Plate539"
files = os.listdir(directory)

file_interest = [file for file in files if file.split('_')[-1]!='kymos' and file.split('.')[-1]!='tiff']
file_interest[34:]

['20220104_Plate539_38',
 '20220104_Plate539_42',
 '20220104_Plate539_35',
 '20220104_Plate539_48',
 '20220104_Plate539_07',
 '20220104_Plate539_70',
 '20220104_Plate539_94',
 '20220104_Plate539_51',
 '20220104_Plate539_26',
 '20220104_Plate539_69',
 '20220104_Plate539_87',
 '20220104_Plate539_14',
 '20220104_Plate539_63',
 '20220104_Plate539_59',
 '20220104_Plate539_53',
 '20220104_Plate539_24',
 '20220104_Plate539_85',
 '20220104_Plate539_16',
 '20220104_Plate539_61',
 '20220104_Plate539_40',
 '20220104_Plate539_37',
 '20220104_Plate539_05',
 '20220104_Plate539_72',
 '20220104_Plate539_96',
 '20220104_Plate539_78',
 '20220104_Plate539_75',
 '20220104_Plate539_02',
 '20220104_Plate539_08',
 '20220104_Plate539_91',
 '20220104_Plate539_30',
 '20220104_Plate539_47',
 '20220104_Plate539_82',
 '20220104_Plate539_88',
 '20220104_Plate539_66',
 '20220104_Plate539_11',
 '20220104_Plate539_29',
 '20220104_Plate539_23',
 '20220104_Plate539_54',
 '20220104_Plate539_27',
 '20220104_Plate539_50',


In [ ]:
plt.style.use('presentation.mplstyle')

for file in file_interest[34:]:
    images_path = os.path.join(directory,file)    # images_path = r"/mnt/sun/home-folder/cargill/20221109_Plate462/20221109_Plate462_006"
    fps = 20
    time_pixel_size = 1/fps #s.pixel
    binning = 2
    magnification = 50
    space_pixel_size = 2*1.725/(magnification)*binning #um.pixel
    video_name = images_path.split('/')[-1]
    kymos_path = '/'.join(images_path.split('/')[:-1]+["_".join((video_name,'kymos'))])
    if not os.path.exists(kymos_path):
        os.mkdir(kymos_path)
    files = os.listdir(images_path)
    images_total_path = [os.path.join(images_path,file) for file in files]
    images_total_path.sort()


    selection_file = images_total_path
    selection_file.sort()
    begin  = 1
    end = -1
    image = imageio.imread(selection_file[end])
    image2 = imageio.imread(selection_file[begin])
    selection_file = selection_file[begin:end]

    fig, ax = plt.subplots()
    ax.imshow(image,cmap="gray")
    ax.imshow(image2,alpha = 0.5)


    segmented,nx_graph_pruned,pos = segment_brightfield(image,thresh = 5e-07)
    fig, ax = plt.subplots()
    ax.imshow(segmented,cmap="gray")

    fig, ax = plt.subplots()
    ax.imshow(image)
    weight = 0.05
    for edge in list(nx_graph_pruned.edges):
        ax.text(*np.flip((1-weight) * pos[edge[0]]+weight*pos[edge[1]]),str(edge[0]),color="white")
        ax.text(*np.flip((1-weight) * pos[edge[1]]+weight*pos[edge[0]]),str(edge[1]),color="white")

    edges = list(nx_graph_pruned.edges)
    edge_oriented = []
    for edge in edges:
        if pos[edge[0]][0]>pos[edge[1]][0]:
            edge_oriented.append(edge)
        else:
            edge_oriented.append((edge[1],edge[0]))
    edges = edge_oriented

    edge_oriented

    np.linalg.norm(pos[edge[0]]-pos[edge[1]])

    fig, ax = plt.subplots()
    ax.imshow(image)
    bound1 = 0
    bound2 = 1
    step=30
    target_length=130
    resolution = 1
    right_edges = []
    for edge in edges:
        offset=int(np.linalg.norm(pos[edge[0]]-pos[edge[1]]))//4
        if offset >=step:
            right_edges.append(edge)
            slices, segments = extract_section_profiles_for_edge(
            edge,
            pos,
            image,
            nx_graph_pruned,
            resolution=resolution,
            offset=offset,
            step=step,
            target_length=target_length,
            bound1=bound1,
            bound2=bound2
        )
            # plot_segments_on_image(segments,ax,color=None)
            plot_segments_on_image(segments,ax, bound1=bound1,
            bound2=bound2,color = 'white',alpha = 0.1)
            ax.plot([pos[edge[0]][1],pos[edge[1]][1]],[pos[edge[0]][0],pos[edge[1]][0]])
            ax.text(*np.flip((1-weight) * pos[edge[0]]+weight*pos[edge[1]]),str(edge[0]),color="white")
            ax.text(*np.flip((1-weight) * pos[edge[1]]+weight*pos[edge[0]]),str(edge[1]),color="white")
    save_path_temp = os.path.join(kymos_path, f"extraction.png")
    plt.savefig(save_path_temp)
    edges = right_edges
    kymos = {edge:get_kymo(edge,pos,selection_file,nx_graph_pruned, resolution=1,
        offset=offset,
        step=step,
        target_length=target_length,
        bound1=bound1,
        bound2=bound2) for edge in edges}
    filtered_kymos = {edge: filter_kymo(kymos[edge]) for edge in edges}
    speeds = {}
    for edge in kymos.keys():
        kymo = kymos[edge]
        save_path_temp = os.path.join(kymos_path, f"{edge}kymo.npy")
        np.save(save_path_temp,kymo)
        im = Image.fromarray(kymo.astype(np.uint8))
        save_path_temp = os.path.join(kymos_path, f"{edge}kymo.png")
        im.save(save_path_temp)

    for edge in kymos.keys():
        for i in [0,1]:
            kymo = filtered_kymos[edge][i]
            suffix = 'right' if i==0 else "left"
            save_path_temp = os.path.join(kymos_path, f"{edge}kymo_{suffix}.npy")
            np.save(save_path_temp,kymo)
            im = Image.fromarray(((kymo-np.min(kymo))/np.max(kymo)*128).astype(np.uint8))
            save_path_temp = os.path.join(kymos_path, f"{edge}kymo_{suffix}.png")
            im.save(save_path_temp)

    fig, ax = plt.subplots()
    speed_dataframe = pd.DataFrame()
    for j,edge in enumerate(edges):
        for i in [0,1]:
            kymo = filter_kymo(kymos[edge])[i]
            W = int(5/space_pixel_size)          # window size is WxW
            W = 3
            C_Thr = 0.95
            imgCoherency, imgOrientation = calcGST(kymo, W)
            nans = np.empty(imgOrientation.shape)
            nans.fill(np.nan)
            real_movement = np.where(imgCoherency>C_Thr,imgOrientation,nans)
            speed=np.tan((real_movement-90)/180*np.pi)*space_pixel_size/time_pixel_size #um.s-1
            nans = np.empty(speed.shape)
            nans.fill(np.nan)
            speed = np.where(speed<20,speed,nans)
            nans = np.empty(speed.shape)
            nans.fill(np.nan)
            speed = np.where(speed>-20,speed,nans)
            # # ax.imshow((imgCoherency))
            klen = 25
            kernel = np.ones((klen,klen))/klen**2
            z1 = scipy.signal.convolve2d(imgCoherency, kernel,mode = "same")
            nans = np.empty(speed.shape)
            nans.fill(np.nan)
            speed = np.where(z1>0.8,speed,nans)
            label =edge if i==0 else None
            times = np.array(range(len(np.mean(speed,axis=1))))*time_pixel_size
            speeds = np.nanmean(speed,axis=1)
            edges_list = [edge for k in range(len(speeds))]
            direction = ["root" if i==0 else 'tip' for k in range(len(speeds))]
            data = pd.DataFrame(np.transpose((times,speeds,edges_list,direction)),
                         columns = ['time (s)','speed (um.s-1)','edge','direction'])
            speed_dataframe = pd.concat((speed_dataframe,data))
            p = ax.plot(times,speeds,label =edge if i==0 else None,color=None if i==0 else color)
            color = p[0].get_color()
            ax.set_ylabel('speed($\mu m.s^{-1}$)')
            ax.set_xlabel('time ($s$)')
    save_path_temp = os.path.join(kymos_path, f"speed_data.csv")
    speed_dataframe.to_csv(save_path_temp)
    plt.legend()
    plt.tight_layout()
    # break
    plt.close('all')

/data/temp/ipykernel_2995333/2001701496.py:23: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(selection_file[end])
/data/temp/ipykernel_2995333/2001701496.py:24: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image2 = imageio.imread(selection_file[begin])
/data/temp/ipykernel_2995333/2001701496.py:143: RuntimeWarning: Mean of empty slice
  speeds = np.nanmean(speed,axis=1)
/home/ipausers/bisot/miniconda3/envs/amftrack/lib/python3.9/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a l